In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path = "/content/gdrive/MyDrive/Hackathon_EFI/Clean Data/data_model.csv"
df = pd.read_csv(path)

In [ ]:
df["Date"] = pd.to_datetime(df.Date)
df.set_index(["Ticker", "Date"], inplace=True)

In [ ]:
df.head()

carbone emission  distance to default  market cap  \
Ticker        Date                                                            
NEM US Equity 2012-03-31            3360.0              8.31311  23133.4917   
              2013-03-31            4260.0              7.93557  11467.7674   
              2014-03-31            4490.0              4.05449   9427.2379   
              2015-03-31            4100.0              4.96930   9518.8237   
              2016-03-31            3800.0              4.58135  18082.6546   

                          operating margin  total assets  working capital  \
Ticker        Date                                                          
NEM US Equity 2012-03-31           19.7142       26111.0           3521.0   
              2013-03-31           31.2627       29650.0           2974.0   
              2014-03-31          -43.4039       24607.0           2363.0   
              2015-03-31           14.5623       24916.0           3317.0   
              2016-03-31            7.5103       25130.0           3322.0   

                          retained earnings  debt ratio  volatility  \
Ticker        Date                                                    
NEM US Equity 2012-03-31            13.9049     21.2355    0.016910   
              2013-03-31            15.9553     21.6937    0.014054   
              2014-03-31             3.0769     27.2643    0.021378   
              2015-03-31             3.5974     25.2012    0.021290   
              2016-03-31             4.3294     23.2314    0.027001   

                          carbone intensity  ...  log total assets  \
Ticker        Date                           ...                     
NEM US Equity 2012-03-31           324.3869  ...         10.170150   
              2013-03-31           431.6984  ...         10.297251   
              2014-03-31           533.6344  ...         10.110827   
              2015-03-31           601.2612  ...         10.123306   
              2016-03-31           624.4864  ...         10.131857   

                          retained earnings ratio  working capital ratio  \
Ticker        Date                                                         
NEM US Equity 2012-03-31                 0.000533               0.134847   
              2013-03-31                 0.000538               0.100304   
              2014-03-31                 0.000125               0.096030   
              2015-03-31                 0.000144               0.133127   
              2016-03-31                 0.000172               0.132193   

                          log carbone emission ante  \
Ticker        Date                                    
NEM US Equity 2012-03-31                   8.119994   
              2013-03-31                   8.357259   
              2014-03-31                   8.409831   
              2015-03-31                   0.000000   
              2016-03-31                   0.000000   

                          log carbone emission post  \
Ticker        Date                                    
NEM US Equity 2012-03-31                   0.000000   
              2013-03-31                   0.000000   
              2014-03-31                   0.000000   
              2015-03-31                   8.318986   
              2016-03-31                   8.243019   

                          log carbone intensity ante  \
Ticker        Date                                     
NEM US Equity 2012-03-31                    5.785015   
              2013-03-31                    6.070041   
              2014-03-31                    6.281583   
              2015-03-31                    0.000000   
              2016-03-31                    0.000000   

                          log carbone intensity post  event        Country  \
Ticker        Date                                                           
NEM US Equity 2012-03-31                    0.000000      0  UNITED STATES   
           

In [ ]:
import statsmodels.api as sm
from linearmodels.panel import PooledOLS
from linearmodels import PanelOLS
from sklearn.preprocessing import StandardScaler

In [ ]:
control_variables = [
    "log total assets", 
    "operating margin", 
    "working capital ratio", 
    "retained earnings ratio",
    "debt ratio",
    "volatility",
    "Oil",
    #"Natural Gas",
    ]
target = ["distance to default"]

filtre = df.Industry == "Energy"

exog = df.loc[filtre, control_variables + ["log carbone emission"]]

autoscaler = StandardScaler()
scaled_exog = autoscaler.fit_transform(exog)
scaled_exog = pd.DataFrame(scaled_exog, index=exog.index, columns=exog.columns)

scaled_exog = sm.tools.tools.add_constant(scaled_exog)
exog = sm.tools.tools.add_constant(exog)

endog = df.loc[filtre, target]

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
model_fe = PanelOLS(endog, exog, entity_effects = False) 
fe_res = model_fe.fit(cov_type="robust")

In [ ]:
print(fe_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     distance to default   R-squared:                        0.6912
Estimator:                    PanelOLS   R-squared (Between):              0.6787
No. Observations:                  189   R-squared (Within):               0.6950
Date:                 Sat, Oct 15 2022   R-squared (Overall):              0.6912
Time:                         01:48:24   Log-likelihood                   -397.00
Cov. Estimator:                 Robust                                           
                                         F-statistic:                      50.354
Entities:                          267   P-value                           0.0000
Avg Obs:                        0.7079   Distribution:                   F(8,180)
Min Obs:                        0.0000                                           
Max Obs:                        14.000   F-statistic (robust):             25.226
                

In [ ]:
control_variables = [
    "log total assets", 
    "operating margin", 
    "working capital ratio", 
    "retained earnings ratio",
    "debt ratio",
    "volatility",
    "Oil",
    #"Natural Gas",
    ]
target = ["distance to default"]

filtre = df.Industry == "Energy"

exog = df.loc[filtre, control_variables + ["log carbone emission"]]

autoscaler = StandardScaler()
scaled_exog = autoscaler.fit_transform(exog)
scaled_exog = pd.DataFrame(scaled_exog, index=exog.index, columns=exog.columns)

scaled_exog = sm.tools.tools.add_constant(scaled_exog)
#exog = sm.tools.tools.add_constant(exog)

endog = df.loc[filtre, target]

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit(cov_type="robust",)

In [ ]:
print(fe_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     distance to default   R-squared:                        0.7095
Estimator:                    PanelOLS   R-squared (Between):             -11.525
No. Observations:                  189   R-squared (Within):               0.7095
Date:                 Sat, Oct 15 2022   R-squared (Overall):             -10.036
Time:                         01:49:20   Log-likelihood                   -340.28
Cov. Estimator:                 Robust                                           
                                         F-statistic:                      47.926
Entities:                          267   P-value                           0.0000
Avg Obs:                        0.7079   Distribution:                   F(8,157)
Min Obs:                        0.0000                                           
Max Obs:                        14.000   F-statistic (robust):             27.140
                

In [ ]:
control_variables = [
    "log total assets", 
    "operating margin", 
    "working capital ratio", 
    "retained earnings ratio",
    "debt ratio",
    "volatility",
    "Oil",
    #"Natural Gas",
    ]
target = ["distance to default"]

filtre = df.Industry == "Energy"

exog = df.loc[filtre, control_variables + ["log carbone intensity"]]

autoscaler = StandardScaler()
scaled_exog = autoscaler.fit_transform(exog)
scaled_exog = pd.DataFrame(scaled_exog, index=exog.index, columns=exog.columns)

scaled_exog = sm.tools.tools.add_constant(scaled_exog)
exog = sm.tools.tools.add_constant(exog)

endog = df.loc[filtre, target]

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit(cov_type="robust")

In [ ]:
print(fe_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     distance to default   R-squared:                        0.7094
Estimator:                    PanelOLS   R-squared (Between):              0.5470
No. Observations:                  189   R-squared (Within):               0.7094
Date:                 Sat, Oct 15 2022   R-squared (Overall):              0.6486
Time:                         02:00:58   Log-likelihood                   -340.31
Cov. Estimator:                 Robust                                           
                                         F-statistic:                      47.899
Entities:                          267   P-value                           0.0000
Avg Obs:                        0.7079   Distribution:                   F(8,157)
Min Obs:                        0.0000                                           
Max Obs:                        14.000   F-statistic (robust):             26.682
                

In [ ]:
control_variables = [
    "log total assets", 
    "operating margin", 
    "working capital ratio", 
    "retained earnings ratio",
    "debt ratio",
    "volatility",
    "Oil",
    #"Natural Gas",
    ]
target = ["distance to default"]

filtre = df.Industry == "Energy"

exog = df.loc[filtre, control_variables + ["log carbone emission ante", "log carbone emission post", "event"]]

autoscaler = StandardScaler()
scaled_exog = autoscaler.fit_transform(exog)
scaled_exog = pd.DataFrame(scaled_exog, index=exog.index, columns=exog.columns)

scaled_exog = sm.tools.tools.add_constant(scaled_exog)
exog = sm.tools.tools.add_constant(exog)

endog = df.loc[filtre, target]

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
model_fe = PanelOLS(endog, exog, entity_effects = True) 
fe_res = model_fe.fit(cov_type="robust")

In [ ]:
print(fe_res)

                           PanelOLS Estimation Summary                           
Dep. Variable:     distance to default   R-squared:                        0.7127
Estimator:                    PanelOLS   R-squared (Between):              0.4018
No. Observations:                  189   R-squared (Within):               0.7127
Date:                 Sat, Oct 15 2022   R-squared (Overall):              0.5984
Time:                         02:02:18   Log-likelihood                   -339.22
Cov. Estimator:                 Robust                                           
                                         F-statistic:                      38.450
Entities:                          267   P-value                           0.0000
Avg Obs:                        0.7079   Distribution:                  F(10,155)
Min Obs:                        0.0000                                           
Max Obs:                        14.000   F-statistic (robust):             24.672
                